### Pytorch Model

Lager en enkel modell med 2 lineære lag

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class Model(nn.Module):
    def __init__(self, input_shape=49, output_shape=9, lr=0.001):
        super(Model, self).__init__()
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.lr = lr
        self.net = torch.nn.Sequential(
            torch.nn.Linear(input_shape, 25),
            torch.nn.ReLU(),
            torch.nn.Linear(25, output_shape),
        )
        self.opt = optim.SGD(self.net.parameters(), lr=lr, momentum=0.9)
        #self.opt = optim.Adam(self.net.parameters(), lr=lr)
        #self.opt = optim.RMSprop(self.net.parameters(), lr=lr)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.net(x)
    
    def accuracy(self, x, y):
        return torch.mean(torch.eq(self.forward(x).argmax(1), y.argmax(1)).float())

    """
    # Uses Cross Entropy
    def loss(self, x, y):
        return torch.nn.functional.binary_cross_entropy_with_logits(self.f(x), y)
    """
    def loss(self, x, y):
        return self.criterion(self.forward(x), y)


### Training

Denne er hentet ut fra [Pytorch egen nettside](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)

In [ ]:
LR = 0.001
EPOCH = 100
AMOUNT_OF_SENSORS = 3

model = Model(input_shape=(AMOUNT_OF_SENSORS*16)+1, output_shape=len(POSE_MAP), lr=LR)

#Vi må dele opp i rader og skille ut en 7tensor med dataattributter og en tensor med labels
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(x_train, 0):
        # zero the parameter gradients
        model.opt.zero_grad()

        # forward + backward + optimize
        outputs = model.forward(inputs)
        loss = model.criterion(outputs, labels)
        loss.backward()
        model.opt.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

results = []
for index, epoch in enumerate(range(EPOCH)):
    model.opt.zero_grad()

    # forward + backward + optimize
    loss = model.loss(torch.tensor(x_train.values), torch.tensor(y_train.values))
    loss.backward()
    model.opt.step()

    if (index+1) % 10 == 0:
        print(f'epoch = {index+1}, loss = {model.loss(x_train, y_train).item()}, accuracy = {model.accuracy(x_test, y_test).item() * 100}%')
        results.append([index+1, model.loss(x_train, y_train).item(),
                        model.accuracy(x_test, y_test).item() * 100])

    model.loss(x_train, y_train).backward()  # Compute loss gradients
    model.opt.step()  # Perform optimization by adjusting W and b

print(tabulate.tabulate(results, headers=['epoch', 'loss', 'accuracy']))